# Exploring Traffic Data

The first thing to do is to check the features of the traffic dataset.

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Download json file

`request` json file with an API.
Download cameras, active segments.

In [2]:
from pandas.io.json import json_normalize
# camera data
cameras = requests.get("https://telraam-api.net/v0/cameras")
nbr_cameras = len(cameras.json()['cameras'][:])
print('Nbr of cameras:',nbr_cameras)
data_cameras = json_normalize(cameras.json()['cameras'])
print(list(data_cameras.columns))
print(data_cameras.head(20))
# check unique segments
data_cameras['segment_id'] = data_cameras['segment_id'].astype(np.uint64)
uniqueSegments = data_cameras['segment_id'].unique()
uniqueSegments.sort()
print(uniqueSegments[-10:-1])

Nbr of cameras: 971
['bikes_left', 'bikes_right', 'cars_left', 'cars_right', 'direction', 'last_data_package', 'mac', 'manual', 'pedestrians_left', 'pedestrians_right', 'segment_id', 'status', 'time_added', 'user_id']
    bikes_left  bikes_right  cars_left  cars_right  direction  \
0        False         True       True        True       True   
1         True         True       True        True      False   
2         True         True       True        True      False   
3         True         True       True        True      False   
4         True         True       True        True      False   
5         True         True       True        True       True   
6         True        False       True        True       True   
7         True         True       True        True      False   
8         True         True       True        True       True   
9        False         True       True        True       True   
10        True         True       True        True       True   
11

In [3]:
# segments data
segments = requests.get("https://telraam-api.net/v0/segments/active")
nbr_segments = len(segments.json()['features'][:])
print('Nbr of segments:',nbr_segments)
data_segments = json_normalize(segments.json()['features'])
print(list(data_segments.columns))
print(data_segments.head(10))
print(data_segments['properties.typical_data'][1])
# sort segment data wrt IDs
data_segments = data_segments.sort_values(['properties.id'], ascending=[True])
activeSegmentIDs = data_segments['properties.id'].values
uniqueActiveSegmentsIDs = data_segments['properties.id'].unique()
print('Number of active segments:'+str(len(activeSegmentIDs)))
print('Number of unique active segments:'+str(len(uniqueActiveSegmentsIDs)))

Nbr of segments: 818
['geometry.coordinates', 'geometry.type', 'properties.bike', 'properties.bike_avg', 'properties.car', 'properties.car_avg', 'properties.car_speed_buckets', 'properties.car_speed_histogram', 'properties.id', 'properties.last_data_package', 'properties.lorry', 'properties.lorry_avg', 'properties.oneway', 'properties.pedestrian', 'properties.pedestrian_avg', 'properties.road_speed', 'properties.road_type', 'properties.speed', 'properties.typical_data', 'type']
                                geometry.coordinates    geometry.type  \
0  [[[4.584397821453392, 51.12834082089698], [4.5...  MultiLineString   
1  [[[4.672590795962118, 52.11733000219071], [4.6...  MultiLineString   
2  [[[4.844638195731497, 50.327054302724086], [4....  MultiLineString   
3  [[[3.9993072959552927, 50.45575610258816], [3....  MultiLineString   
4  [[[15.448795992214585, 47.06951620560343], [15...  MultiLineString   
5  [[[15.448795992214585, 47.06951620560343], [15...  MultiLineString   
6  [[[

## Clean inactive segment from camera datasets

It shows that there are only 764 unique segments in the data. The duplicate segments might vary in terms of direction. It is still a question to be answered.

In [4]:
inactiveSegmentIDs = list(set(uniqueSegments) ^ set(activeSegmentIDs))
print('Inactive segments = '+ str(len(inactiveSegmentIDs)))
commonSegmentIDs = set(uniqueSegments).intersection(activeSegmentIDs)
print('Common segments = ' + str(len(commonSegmentIDs)))
print('If active == unique segments:')
if len(inactiveSegmentIDs)+len(activeSegmentIDs) == len(uniqueSegments):
    print('\t' + str(True))
else:
    print('\t' + str(False))

Inactive segments = 83
Common segments = 765
If active == unique segments:
	False


## Timestamps

Date and hour registration. To check if they need sorting again.

In [5]:
# Camera data timestamps
dateveri = data_cameras['time_added'].values
hours = pd.to_datetime(np.squeeze(dateveri)).hour
dates = pd.to_datetime(np.squeeze(dateveri)).date
# camera data needs sorting again
data_cameras = data_cameras.sort_values(['status','segment_id','time_added'], ascending=[True, True, True])
print(data_cameras.head(10))


     bikes_left  bikes_right  cars_left  cars_right  direction  \
20         True         True       True        True       True   
14         True         True       True        True      False   
726        True         True       True        True      False   
724        True         True       True        True      False   
861        True         True       True        True       True   
673        True         True       True        True      False   
471        True         True       True        True       True   
582        True         True       True        True      False   
855        True         True       True        True      False   
454        True         True       True        True       True   

            last_data_package              mac  manual  pedestrians_left  \
20   2020-02-17T20:17:34.587Z  202481588641665   False              True   
14   2020-02-17T20:17:34.587Z  202481588641665   False              True   
726  2020-02-17T20:16:53.574Z  20248158781019

In [6]:
# Segment data timestamps
dateveri = data_segments['properties.typical_data'].values
print(dateveri[0])

# hours = pd.to_datetime(np.squeeze(dateveri)).hour
# dates = pd.to_datetime(np.squeeze(dateveri)).date
# camera data needs sorting again

[{'hour': '05:00:00', 'pedestrian': None, 'bike': None, 'car': None, 'lorry': None}, {'hour': '06:00:00', 'pedestrian': None, 'bike': None, 'car': None, 'lorry': None}, {'hour': '07:00:00', 'pedestrian': None, 'bike': None, 'car': None, 'lorry': None}, {'hour': '08:00:00', 'pedestrian': 0, 'bike': 4, 'car': 105, 'lorry': 0}, {'hour': '09:00:00', 'pedestrian': 1, 'bike': 33, 'car': 141, 'lorry': 2}, {'hour': '10:00:00', 'pedestrian': 1, 'bike': 23, 'car': 202, 'lorry': 6}, {'hour': '11:00:00', 'pedestrian': 1, 'bike': 3, 'car': 180, 'lorry': 6}, {'hour': '12:00:00', 'pedestrian': 1, 'bike': 19, 'car': 192, 'lorry': 4}, {'hour': '13:00:00', 'pedestrian': 1, 'bike': 10, 'car': 203, 'lorry': 5}, {'hour': '14:00:00', 'pedestrian': 0, 'bike': 11, 'car': 211, 'lorry': 4}, {'hour': '15:00:00', 'pedestrian': 0, 'bike': 0, 'car': 141, 'lorry': 0}, {'hour': '16:00:00', 'pedestrian': 2, 'bike': 15, 'car': 330, 'lorry': 0}, {'hour': '17:00:00', 'pedestrian': None, 'bike': None, 'car': None, 'lorry'

## Position of Segments

We only take the middle position of the segments.

In [9]:
segmentCoordinates = np.zeros((len(activeSegmentIDs),2))
# Read coordinates from each segment
for i in range(len(activeSegmentIDs)):
    urlSegment = 'https://telraam-api.net/v0/segments/id/'+str(activeSegmentIDs[i])
    active = requests.get(urlSegment)
    co = np.array(active.json()['features'][0]['geometry']['coordinates'][0]) # coordinates of the cameras
    segmentCoordinates[i,:] = np.mean(co, axis=0)
    
print(segmentCoordinates[0:2,:])

[[ 5.16331069 50.8910606 ]
 [ 5.11286407 50.89409181]]


In [10]:
# Restrict them to Leuven coordinates
activeSegmentIDs = np.expand_dims(np.array(activeSegmentIDs),axis=-1)
segmentCoordinates = np.concatenate((activeSegmentIDs,segmentCoordinates),axis=1)
segmentCoordinates = pd.DataFrame(data=segmentCoordinates, columns=['SegmentID', 'LON', 'LAT'])
print(segmentCoordinates.head(10))

   SegmentID       LON        LAT
0    20499.0  5.163311  50.891061
1    20580.0  5.112864  50.894092
2    24948.0  4.476654  51.302504
3    26137.0  4.487590  51.299626
4    26358.0  3.728146  51.033024
5    26380.0  3.724681  51.032293
6    26408.0  3.706345  51.035715
7    26683.0  3.694862  51.035912
8    26940.0  3.755602  51.045376
9    27009.0  3.758483  51.040428


In [11]:
segmentCoordinates = segmentCoordinates.astype({'SegmentID': 'uint64'})
print(segmentCoordinates.head(10))

   SegmentID       LON        LAT
0      20499  5.163311  50.891061
1      20580  5.112864  50.894092
2      24948  4.476654  51.302504
3      26137  4.487590  51.299626
4      26358  3.728146  51.033024
5      26380  3.724681  51.032293
6      26408  3.706345  51.035715
7      26683  3.694862  51.035912
8      26940  3.755602  51.045376
9      27009  3.758483  51.040428


In [12]:
# Leuven Coordinates
LeuvenCoord = np.array([[4.654269, 4.789127],[50.846792,50.937804]])
LeuvenSegmentCoordinates = segmentCoordinates[segmentCoordinates['LON']>=LeuvenCoord[0,0]]
LeuvenSegmentCoordinates = segmentCoordinates[segmentCoordinates['LON']<=LeuvenCoord[0,1]]
LeuvenSegmentCoordinates = segmentCoordinates[segmentCoordinates['LAT']>=LeuvenCoord[1,0]]
LeuvenSegmentCoordinates = segmentCoordinates[segmentCoordinates['LAT']>=LeuvenCoord[1,1]]
print(LeuvenSegmentCoordinates.shape)

(428, 3)


## Finding most reliable segment

In [15]:
# read segment 1-by-1
LeuvenSegmentIDs = LeuvenSegmentCoordinates['SegmentID'].values
counterSegments = np.zeros_like(LeuvenSegmentIDs)
# url to read a single segment 
url_base = 'https://telraam-api.net/v0/cameras/segment/'
# number of cameras per segment
for  i  in np.arange(0,len(LeuvenSegmentIDs)):
    url = url_base + str(LeuvenSegmentIDs[i])
    telraam_segment = requests.get(url)
    c = int(len(telraam_segment.json()['camera']))
    counterSegments[i] = c

In [17]:
start = 0; stop = 10;
print(counterSegments[start:stop])
# sort them with respect to the number of cameras
countdf = pd.DataFrame(counterSegments,index=LeuvenSegmentIDs,columns=['Nbr'])
countdf.sort_values(by='Nbr',inplace=True,ascending=False)
relSegID = countdf.index[0]

[2 1 1 1 1 1 1 1 1 1]


In [19]:
countdf.head(4)
print(relSegID)

122431


In [ ]:
LeuvenSegmentCoordinates